 We have the following data fields in our dataset:

* N - ratio of Nitrogen content in soil
* P - ratio of Phosphorous content in soil
* K - ratio of Potassium content in soil
* temperature - temperature in degree Celsius
* humidity - relative humidity in %
* ph - ph value of the soil
* rainfall - rainfall in mm


Let's begin with importing the required libraries.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dropout
import matplotlib.pyplot as plt


We read the dataset and convert it into a dataframe.

In [3]:
path_train = r"data\Crop_recommendation.csv"
train_data = pd.DataFrame(pd.read_csv(path_train))
print(train_data.head())

    N   P   K  temperature   humidity        ph    rainfall label
0  90  42  43    20.879744  82.002744  6.502985  202.935536  rice
1  85  58  41    21.770462  80.319644  7.038096  226.655537  rice
2  60  55  44    23.004459  82.320763  7.840207  263.964248  rice
3  74  35  40    26.491096  80.158363  6.980401  242.864034  rice
4  78  42  42    20.130175  81.604873  7.628473  262.717340  rice


Let's see how many possible categories exist in the label column(the column we are going to predict)

In [4]:
print(train_data['label'].unique())
classes = train_data['label'].unique()

['rice' 'maize' 'chickpea' 'kidneybeans' 'pigeonpeas' 'mothbeans'
 'mungbean' 'blackgram' 'lentil' 'pomegranate' 'banana' 'mango' 'grapes'
 'watermelon' 'muskmelon' 'apple' 'orange' 'papaya' 'coconut' 'cotton'
 'jute' 'coffee']


Next, we will prepare our data for trainaing the neural network(we need X_train and Y_train)

In [5]:
Y_train = train_data['label']
X_train = train_data.drop('label', axis = 1)
print(X_train.head())
print("")
print(Y_train.head())

    N   P   K  temperature   humidity        ph    rainfall
0  90  42  43    20.879744  82.002744  6.502985  202.935536
1  85  58  41    21.770462  80.319644  7.038096  226.655537
2  60  55  44    23.004459  82.320763  7.840207  263.964248
3  74  35  40    26.491096  80.158363  6.980401  242.864034
4  78  42  42    20.130175  81.604873  7.628473  262.717340

0    rice
1    rice
2    rice
3    rice
4    rice
Name: label, dtype: object


Perfect, now we need to encode all categorical variables we have (Y_train). I made use of OneHotEncoder.Observe that instead of the name of a crop, now we have a hot vector with 22 columns,each column corresponding to a crop type.

In [6]:
Encoder = OneHotEncoder(handle_unknown='ignore')

Y_train = pd.DataFrame(Encoder.fit_transform(train_data[['label']]).toarray())

print(Y_train)

       0    1    2    3    4    5    6    7    8    9   ...   12   13   14  \
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2195  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2196  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2197  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2198  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2199  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

       15   16   17   18   19   20   21  
0     0.0  0.0  0.0  

Another important part is scaling our data before inputing it in our nn.

In [7]:
sc = StandardScaler()
train_scaled = sc.fit_transform(X_train)
print(train_scaled)
data = train_scaled

[[ 1.0687974  -0.34455075 -0.1016875  ...  0.47266646  0.04330173
   1.8103605 ]
 [ 0.93332887  0.14061552 -0.14118477 ...  0.39705125  0.73487256
   2.24205791]
 [ 0.25598625  0.04964684 -0.08193887 ...  0.48695381  1.77151047
   2.92106603]
 ...
 [ 1.82742114 -0.61745677 -0.35841972 ... -0.19123516 -0.13812031
   1.27141766]
 [ 1.80032743 -0.64777967 -0.27942519 ... -0.86951801  0.37390383
   0.43154519]
 [ 1.44810927 -1.07230015 -0.35841972 ... -0.49802006  0.40109573
   0.68200518]]


We create the neural network model and begin training.

In [8]:
model = Sequential()
model.add(Dense(1000, activation='relu', input_shape=(data.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(500, activation='tanh'))
model.add(Dense(22, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train,Y_train, epochs = 100)

Epoch 1/100
69/69 [==============================] - 3s 4ms/step - loss: 1.5510 - accuracy: 0.5305
Epoch 2/100
69/69 [==============================] - 0s 4ms/step - loss: 0.5878 - accuracy: 0.8082
Epoch 3/100
69/69 [==============================] - 0s 4ms/step - loss: 0.3549 - accuracy: 0.8936
Epoch 4/100
69/69 [==============================] - 0s 5ms/step - loss: 0.2769 - accuracy: 0.9091
Epoch 5/100
69/69 [==============================] - 0s 4ms/step - loss: 0.2061 - accuracy: 0.9377
Epoch 6/100
69/69 [==============================] - 0s 3ms/step - loss: 0.1924 - accuracy: 0.9341
Epoch 7/100
69/69 [==============================] - 0s 5ms/step - loss: 0.1851 - accuracy: 0.9345
Epoch 8/100
69/69 [==============================] - 0s 4ms/step - loss: 0.1905 - accuracy: 0.9309
Epoch 9/100
69/69 [==============================] - 0s 4ms/step - loss: 0.1633 - accuracy: 0.9409
Epoch 10/100
69/69 [==============================] - 0s 6ms/step - loss: 0.1599 - accuracy: 0.9432
Epoch 11/

Great accuracy! Feel free to try test it and see how accurate 

## Make predictions :

In [9]:
classes[np.argmax(model.predict(X_train.iloc[198:199, :]))]

1/1 [==============================] - 0s 143ms/step


'mango'

In [14]:
# make prediction function
def predict_crop(N, P, K, temperature, humidity, ph, rainfall):
    input = np.array([[N, P, K, temperature, humidity, ph, rainfall]])
    input_scaled = sc.transform(input)
    # print(input_scaled)
    print(classes[np.argmax(model.predict(input_scaled))])


In [18]:
Nitrogen_Ratio = 69
phosphorous_Ratio = 42
potassium_Ratio = 24
temperature = 20
humidity = 80
ph = 6.5
rainfall = 200

predict_crop(Netrogen_Ratio, phosphorous_Ratio, potassium_Ratio, temperature, humidity, ph, rainfall)

1/1 [==============================] - 0s 22ms/step
jute


c:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
